# Chapter 12.8: French verb phrases

__NOTE: MANY CODE SNIPPETS FOR PRODUCTION TAKE A LONG TIME TO EVALUATE__

In [ ]:
using DataFrames, JudiLing

Loading the data

In [ ]:
french = JudiLing.load_dataset("../dat/french.csv");

In [ ]:
size(french)

In [ ]:
first(french, 10)

## Di-syllables

### Full dataset

Creating C and S matrices

In [ ]:
cue_obj = JudiLing.make_cue_matrix(french,
                                   grams=2,
                                   tokenized=true,
                                   sep_token="-",
                                   keep_sep=true,
                                   target_col = :Syllables);

In [ ]:
S = JudiLing.make_S_matrix(french,
     ["Lexeme"],
     ["Person", "Number", "Gender", "Tense", "Aspect", "Class", "Mood"],
     ncol=1000)

Comprehension

In [ ]:
F = JudiLing.make_transform_matrix(cue_obj.C, S);
Shat = cue_obj.C * F;
JudiLing.eval_SC(Shat, S, french, :Syllables)

Production

In [ ]:
G = JudiLing.make_transform_matrix(S, cue_obj.C);
Chat = S * G;
JudiLing.eval_SC(Chat, cue_obj.C, french, :Syllables)

In [ ]:
@time res = JudiLing.learn_paths(french,
                           cue_obj,
                           S,
                           F,
                           Chat,
                           threshold=0.01,
                           Shat_val=Shat, 
                           verbose=true);

321-762 seconds

In [ ]:
JudiLing.eval_acc(res, cue_obj)

### Training and held-out data

Loading the data with careful split.

In [ ]:
data_train, data_val = JudiLing.loading_data_careful_split(
     "../dat/french.csv", "french", "../dat/careful",
     ["Lexeme", "Person", "Number", "Gender", "Tense", "Aspect", "Class", "Mood"],
     n_grams_target_col = "Syllables",
     grams = 2,
     val_sample_size = 2000,
     random_seed = 314)

Creating C and S matrices.

In [ ]:
cue_obj_train, cue_obj_val  = JudiLing.make_combined_cue_matrix(
          data_train,
          data_val,
          grams=2,
          sep_token="-",
          target_col="Syllables");

In [ ]:
S_train, S_val = JudiLing.make_combined_S_matrix(
          data_train,
          data_val,
          ["Lexeme"],
          ["Person", "Number", "Gender", "Tense", "Aspect", "Class", "Mood"],
          ncol=1000);

Comprehension

In [ ]:
F = JudiLing.make_transform_matrix(cue_obj_train.C, S_train);
Shat_val = cue_obj_val.C * F;
@time JudiLing.eval_SC(Shat_val, S_val, S_train)

In [ ]:
@time JudiLing.eval_SC_loose(Shat_val, S_val, S_train, 5)

In [ ]:
@time JudiLing.eval_SC_loose(Shat_val, S_val, S_train, 10)

Production

In [ ]:
G = JudiLing.make_transform_matrix(S_train, cue_obj_train.C);
Chat_val = S_val * G;

In [ ]:
max_t = JudiLing.cal_max_timestep(data_train, data_val, "Syllables", sep_token="-")

In [ ]:
@time prod_val = JudiLing.learn_paths(
        data_train,            
        data_val,              
        cue_obj_train.C,       
        S_val,                 
        F,                     
        Chat_val,              
        cue_obj_val.A,         
        cue_obj_train.i2f,     
        cue_obj_train.f2i,     
        max_t=max_t,
        threshold=0.025,
        grams=2,
        sep_token="-",
        target_col="Syllables",
        verbose=true);

623 seconds

In [ ]:
@time JudiLing.eval_acc(prod_val, cue_obj_val)

Now with tolerance mode on.

In [ ]:
@time prod_val = JudiLing.learn_paths(
        data_train,            
        data_val,              
        cue_obj_train.C,       
        S_val,                 
        F,                     
        Chat_val,              
        cue_obj_val.A,         
        cue_obj_train.i2f,     
        cue_obj_train.f2i,     
        max_t=max_t,
        threshold=0.05,
        grams=2,
        sep_token="-",
        target_col="Syllables",
        is_tolerant = true,
        tolerance = -0.1,
        max_tolerance = 1,  
        verbose=true);

1022 seconds

In [ ]:
JudiLing.eval_acc(prod_val, cue_obj_val)

Running the model with threshold = 0.025 and tolerance mode on takes extremely long to estimate (at least a day), illustrating the limits of what the production algorithm can accomplish for a large dataset with long words.  We expect improved performance with higher thresholds if the Chat matrix is estimated more precisely (for instance, by using the cross-entropy loss).  In what follows, we move from syllables to triphones. 

## Triphones

### Full dataset

Creating C and S matrices

In [ ]:
cue_obj = JudiLing.make_cue_matrix(french,
                                   grams=3,
                                   target_col = :Phon2);
size(cue_obj.C)
# (21360, 3354)

In [ ]:
S = JudiLing.make_S_matrix(french,
     ["Lexeme"],
     ["Person", "Number", "Gender", "Tense", "Aspect", "Class", "Mood"],
     ncol=1000)

Comprehension

In [ ]:
F = JudiLing.make_transform_matrix(cue_obj.C, S);
Shat = cue_obj.C * F;
JudiLing.eval_SC(Shat, S, french, :Phon2)

Production

In [ ]:
G = JudiLing.make_transform_matrix(S, cue_obj.C);
Chat = S * G;
JudiLing.eval_SC(Chat, cue_obj.C, french, :Phon2)

In [ ]:
@time res = JudiLing.learn_paths(french,
                           cue_obj,
                           S,
                           F,
                           Chat,
                           threshold=0.01,
                           Shat_val=Shat, 
                           verbose=true);

528 seconds

In [ ]:
JudiLing.eval_acc(res, cue_obj)

Next, check whether the speech errors are ok, i.e., whether the model gets the liaison right.

In [ ]:
JudiLing.write2csv(res, french, cue_obj, cue_obj,
                          "../res/french_prod.csv", target_col=:Phon2)

### Held-out data

Careful split, this time based on triphones rather than di-syllables

In [ ]:
data_train, data_val = JudiLing.loading_data_careful_split(
     "../dat/french.csv", "french", "../dat/careful",
     ["Lexeme", "Person", "Number", "Gender", "Tense", "Aspect", "Class", "Mood"],
     n_grams_target_col = "Phon2",
     grams = 3,
     val_sample_size = 2000,
     random_seed = 314)

Creating C and S matrices

In [ ]:
cue_obj_train, cue_obj_val  = JudiLing.make_combined_cue_matrix(
          data_train,
          data_val,
          grams=3,
          target_col="Phon2");

In [ ]:
S_train, S_val = JudiLing.make_combined_S_matrix(
          data_train,
          data_val,
          ["Lexeme"],
          ["Person", "Number", "Gender", "Tense", "Aspect", "Class", "Mood"],
          ncol=1000);

Comprehension

In [ ]:
F = JudiLing.make_transform_matrix(cue_obj_train.C, S_train);
Shat_val = cue_obj_val.C * F;
JudiLing.eval_SC(Shat_val, S_val, S_train)

In [ ]:
JudiLing.eval_SC_loose(Shat_val, S_val, S_train, 5)

In [ ]:
JudiLing.eval_SC_loose(Shat_val, S_val, S_train, 10)

Production

In [ ]:
G = JudiLing.make_transform_matrix(S_train, cue_obj_train.C);
Chat_val = S_val * G;

In [ ]:
max_t = JudiLing.cal_max_timestep(data_train, data_val, "Phon2")

In [ ]:
@time prod_val = JudiLing.learn_paths(
        data_train,            
        data_val,              
        cue_obj_train.C,       
        S_val,                 
        F,                     
        Chat_val,              
        cue_obj_val.A,         
        cue_obj_train.i2f,     
        cue_obj_train.f2i,     
        max_t=max_t,
        threshold=0.025,
        grams=3,
        target_col="Phon2",
        verbose=true);

33 seconds

In [ ]:
JudiLing.eval_acc(prod_val, cue_obj_val)

now with tolerance mode on:

In [ ]:
@time prod_val = JudiLing.learn_paths(
        data_train,            
        data_val,              
        cue_obj_train.C,       
        S_val,                 
        F,                     
        Chat_val,              
        cue_obj_val.A,         
        cue_obj_train.i2f,     
        cue_obj_train.f2i,     
        max_t=max_t,
        threshold=0.025,
        grams=3,
        target_col="Phon2",
        is_tolerant = true,
        tolerance = -0.1,
        max_tolerance = 1,  
        verbose=true);

JudiLing.eval_acc(prod_val, cue_obj_val)

125 seconds

In [ ]:
@time prod_val2 = JudiLing.learn_paths(
        data_train,            
        data_val,              
        cue_obj_train.C,       
        S_val,                 
        F,                     
        Chat_val,              
        cue_obj_val.A,         
        cue_obj_train.i2f,     
        cue_obj_train.f2i,     
        max_t=max_t,
        threshold=0.01,
        grams=3,
        target_col="Phon2",
        is_tolerant = true,
        tolerance = -0.1,
        max_tolerance = 1,  
        verbose=true);

JudiLing.eval_acc(prod_val2, cue_obj_val)

304 seconds

## Exercises

Run learn paths on the triphone model without tolerance mode and higher thresholds.

In [ ]:
# Threshold 0.05
@time prod_val3 = JudiLing.learn_paths(
        data_train,            
        data_val,              
        cue_obj_train.C,       
        S_val,                 
        F,                     
        Chat_val,              
        cue_obj_val.A,         
        cue_obj_train.i2f,     
        cue_obj_train.f2i,     
        max_t=max_t,
        threshold=0.05,
        grams=3,
        target_col="Phon2",
        verbose=true);

30.8 seconds

In [ ]:
JudiLing.eval_acc(prod_val3, cue_obj_val)

In [ ]:
# Threshold 0.1
@time prod_val4 = JudiLing.learn_paths(
        data_train,            
        data_val,              
        cue_obj_train.C,       
        S_val,                 
        F,                     
        Chat_val,              
        cue_obj_val.A,         
        cue_obj_train.i2f,     
        cue_obj_train.f2i,     
        max_t=max_t,
        threshold=0.1,
        grams=3,
        target_col="Phon2",
        verbose=true);

29.6 seconds

In [ ]:
JudiLing.eval_acc(prod_val4, cue_obj_val)

With a threshold of 0.025 we had some 95 paths to consider per target word. With a threshold of 0.05, this goes down to about 5 and with a threshold of 0.1, there typically is only one path (or even none). This also strongly affects accuracy, which went down from about 36% with a threshold of 0.025 to about 0.2% with a threshold of 0.1.